<a href="https://colab.research.google.com/github/MPdeM/Final_Project/blob/master/nlp_bert_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Pytorch/ Import Dependencies**

In [0]:
pip install pytorch-pretrained-bert

In [0]:
# Dependencies
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

## **Load/ Clean Data**

In [0]:
# Read in csv file
url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"

fake_jobs_df = pd.read_csv(url)
fake_jobs_df

In [0]:
# Take a look at job description and fraudulent cols
df = fake_jobs_df[['description', 'fraudulent']]
df

In [0]:
# Count how many Real vs Fraudulent postings 
from collections import Counter
print(Counter(df['fraudulent'].values))

In [0]:
# Drop duplicate descriptions
df = df.drop_duplicates()
df

In [0]:
# Count how many Real vs Fraudulent postings 
print(Counter(df['fraudulent'].values))

In [0]:
# Make sure number of real vs fraudulent are more balanced
# Using UNDERsampling for balancing of classification groups
df_fraud= df[df['fraudulent'] == 1] 
df_normal = df[df['fraudulent'] == 0] 

df_normal = df_normal.sample(n=len(df_fraud), random_state=22)
df = df_normal.append(df_fraud)

df

In [0]:
# Shuffle the dataframe so fraudulent postings more evenly distributed 
from sklearn.utils import shuffle
df = shuffle(df, random_state=22)
df = df.reset_index(drop=True)
df

In [0]:
# Explicitly define data types
df['description'] = df['description'].astype(str)
df['fraudulent'] = df['fraudulent'].astype(int)

In [0]:
# Datatypes
df.dtypes

In [0]:
# Count Real vs Fraudulent again to make sure we've achieved the desired result
print(Counter(df['fraudulent'].values))

## **Assign Training/Testing Data**

In [0]:
# Split the data into training and testing
train_data = df.head(200)
test_data = df.tail(150)

print(train_data.shape, test_data.shape)

In [0]:
# Generate a list of dictionaries with ‘description’ and ‘fraudulent’ keys
train_data = [{'description': description, 'fraudulent': fraudulent } for description in list(train_data['description']) for fraudulent in list(train_data['fraudulent'])]

test_data = [{'description': description, 'fraudulent': fraudulent } for description in list(test_data['description'])for fraudulent in list(test_data['fraudulent'])]

In [0]:
# Generate a list of tuples from the list of dictionaries
train_texts, train_labels = list(zip(*map(lambda d: (d['description'], d['fraudulent']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['description'], d['fraudulent']), test_data)))

## **Tokenizer, Token IDs**

In [0]:
# Generate tokens and token ids
# Bert Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
# Tokens
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

In [0]:
# Token Ids
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

In [0]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [0]:
# Generate a boolean array based on the value of ‘fraudulent’ for our testing and training sets
train_y = np.array(train_labels) == 1
test_y = np.array(test_labels) == 1

## **BERT Classifier**

In [0]:
# Create BERT classifier - contains ‘initialization’ method and ‘forward’ method; returns token probabilities
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [0]:
# Generate training and testing masks
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [0]:
# Generate token tensors for training and testing
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [0]:
# Prepare data loaders
BATCH_SIZE = 1

train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

## **Fit the Model using Training Data**

In [0]:
# Use the Adam optimizer to minimize the Binary Cross Entropy loss 
# Train with a batch size of 1 for 1 EPOCHS
BATCH_SIZE = 1
EPOCHS = 1

bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)

for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0

    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

## **Evaluate Model**

In [0]:
# Evaluate our model
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

token_ids, masks, labels = tuple(t for t in batch_data)

logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))